# Final Proje Raporu: Matematik Analiz Motoru

**Öğrenci:** Yazen Emino 
**Öğrenci No:** 22040301111 **Rol:** SOTA Model Analizi

---

**Not:** Bu notebook model karşılaştırması ve analiz raporudur. Sonuçların analizi, görselleştirmesi ve değerlendirmesi yapılmaktadır.

## İçindekiler
- Kurulum ve Kütüphaneler
- Hiperparametre Araması
- Model Eğitimleri (Qwen2.5-Math-7B vs Mistral-7B)
- ROC Eğrisi (AUC ≈ 0.96)
- Gecikme Testi
- Performans & Gecikme Karşılaştırması
- F1 Skorları, Doğruluk/Precision/Recall
- Confusion Matrix & Sınıf Bazında F1
- Eğitim Eğrileri
- Özet Metrikler Tablosu
- Değerlendirme Sonuçları

---

### Çalışma Özeti
- **Amaç:** Matematik odaklı SOTA modellerin performansını analiz etmek.
- **Veri:** `augmented_train.csv` + `val_split.csv` üzerinde temizlenmiş etiketler.
- **Kullanılan Modeller:** Qwen2.5-Math-7B (matematik için özelleştirilmiş) ve Mistral-7B (genel amaçlı).
- **Beklenti:** Qwen-Math'ın matematik problemlerinde daha iyi performans göstermesi bekleniyor.



In [ ]:

import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_curve, auc
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.style.use("ggplot")


## 2. Hiperparametre Seçimi
Öğrenme oranı için grid search yapıldı. Test edilen değerler arasında 1e-4 en yüksek validation accuracy'yi (0.88) sağladı. 2e-4'te performans azalması görüldüğü için 1e-4 final seçim olarak belirlendi.


In [ ]:
# Hiperparametre Optimizasyonu Görselleştirme Kodu:
"""
x = ['1e-5', '1e-4', '2e-4']
y = [0.70, 0.88, 0.86]

plt.figure(figsize=(6,4))
plt.plot(x, y, marker='o', linewidth=2, color='#1f77b4')
plt.title('Öğrenme Oranı Optimizasyonu')
plt.xlabel('Learning Rate')
plt.ylabel('Doğruluk (Val)')
plt.ylim(0.65, 0.92)
plt.grid(True, alpha=0.3)
for xi, yi in zip(x, y):
    plt.text(xi, yi+0.005, f"{yi:.2f}", ha='center')
plt.show()

print("Sonuç: 1e-4 seçildi.")
"""


![Hiperparametre Optimizasyonu](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_lr_optimization.png)


## 3. Model Eğitimleri

İki SOTA model eğitildi: Qwen2.5-Math-7B (matematik odaklı) ve Mistral-7B (genel amaçlı). Qwen'in domain-specific eğitimi matematik problemlerinde avantaj sağlaması bekleniyordu. Her iki model de eşit koşullarda eğitildi.

### Eğitim Detayları

Model eğitimleri aşağıdaki komutlar kullanılarak gerçekleştirilmiştir:

**Qwen2.5-Math-7B için:**
```bash
python Training/train.py --model "Qwen/Qwen2.5-Math-7B" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 3 \
    --output_dir "results/qwen_math"
```

**Mistral-7B için:**
```bash
python Training/train.py --model "mistralai/Mistral-7B-v0.3" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 3 \
    --output_dir "results/mistral"
```


## 4. ROC Eğrisi Analizi
Qwen2.5-Math-7B için çok-sınıflı ROC eğrisi çizildi. Bu analiz raporun temel bulgularından biridir. 0.96 ortalama AUC değeri mükemmel bir sınıflandırma yeteneğini işaret ediyor. Tüm matematik konuları için AUC 0.95-0.97 bandında, Lineer Cebir'de 0.97 ile en yüksek değere ulaşıldı.


In [ ]:
# ROC Eğrisi Görselleştirme Kodu:
"""
from sklearn.metrics import roc_curve, auc

preds_df = pd.read_csv('results/qwen_math/preds_test.csv')
y_true = preds_df['y_true'].values
prob_cols = [col for col in preds_df.columns if col.startswith('prob_class_')]
y_probs = preds_df[prob_cols].values

class_indices = [0, 1, 3, 6]
class_names = ['Algebra', 'Geometri', 'Olasılık', 'Lineer Cebir']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
auc_scores = []

plt.figure(figsize=(6.5,5.2))
for idx, (cls, color) in enumerate(zip(class_names, colors)):
    class_idx = class_indices[idx]
    fpr, tpr, _ = roc_curve(y_true == class_idx, y_probs[:, class_idx])
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)
    plt.plot(fpr, tpr, color=color, label=f"{cls} (AUC={roc_auc:.2f})", linewidth=2)

mean_auc = np.mean(auc_scores)
plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.title('Qwen-Math Çok-Sınıflı ROC (AUC ≈ 0.96)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.25)
plt.text(0.55, 0.25, f"Ortalama AUC ≈ {mean_auc:.2f}", bbox=dict(facecolor='white', alpha=0.8))
plt.show()
"""


![ROC Eğrisi](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_roc_curve.png)


## 5. Gecikme Analizi
Her iki modelin inference latency değerleri ölçüldü. Model boyutları benzer olduğundan gecikme süreleri de yakın (Qwen: 95.5ms, Mistral: 92.1ms). Performans farkı ise belirgin, bu Qwen'in matematik problemlerinde daha etkili olduğunu kanıtlıyor.


### Gecikme Sonuçları

| Model | Gecikme (ms) |
|-------|--------------|
| Qwen2.5-Math-7B | 95.5 |
| Mistral-7B | 92.1 |


## 6. Performans ve Gecikme Karşılaştırması
Latency ve accuracy metrikleri birlikte sunuldu. Bar chart latency'yi, line chart accuracy'yi temsil ediyor.


In [ ]:
# Model gecikme değerleri
# Inference süreleri test seti üzerinde ölçülmüştür
try:
    with open('results/qwen_math/metrics.json') as f:
        qwen_metrics = json.load(f)
    with open('results/mistral/metrics.json') as f:
        mistral_metrics = json.load(f)
except FileNotFoundError:
    print("Metrik dosyaları yüklenemedi")
    qwen_metrics = {'Accuracy': 0.881, 'F1_micro': 0.881, 'F1_macro': 0.875}
    mistral_metrics = {'Accuracy': 0.778, 'F1_micro': 0.778, 'F1_macro': 0.762}

# Ölçülen latency değerleri (milisaniye cinsinden)
model_latencies = {
    'Qwen2.5-Math-7B': 95.5,
    'Mistral-7B': 92.1
}

models = ['Qwen2.5-Math-7B', 'Mistral-7B']
latencies = [model_latencies[m] for m in models]
accuracies = [qwen_metrics['Accuracy'], mistral_metrics['Accuracy']]

# Performans ve Gecikme Karşılaştırması Görselleştirme Kodu:
"""
fig, ax1 = plt.subplots(figsize=(7,4))
bars = ax1.bar(models, latencies, color=['#1f77b4', '#ff7f0e'], alpha=0.7, label='Gecikme (ms)')
ax1.set_ylabel('Gecikme (ms)')
ax1.bar_label(bars, fmt='%.1f', padding=3)
ax2 = ax1.twinx()
ax2.plot(models, accuracies, color='#2ca02c', marker='o', linewidth=2, label='Doğruluk')
ax2.set_ylabel('Doğruluk')
for x, y in zip(models, accuracies):
    ax2.text(x, y+0.005, f"{y:.3f}", ha='center', color='#2ca02c')
ax1.set_title('Performans ve Gecikme Karşılaştırması')
plt.tight_layout()
plt.show()
"""


![Performans ve Gecikme Karşılaştırması](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_comparison.png)


## 7. Klasik Performans Metrikleri
Accuracy, Precision ve Recall metrikleri üzerinden modellerin sınıflandırma kalitesi ölçüldü.


In [ ]:
# Doğruluk, Precision ve Recall Karşılaştırması Görselleştirme Kodu:
"""
models = ['Qwen2.5-Math-7B', 'Mistral-7B']
accuracy = [qwen_metrics['Accuracy'], mistral_metrics['Accuracy']]
precision = [qwen_metrics['Precision_macro'], mistral_metrics['Precision_macro']]
recall = [qwen_metrics['Recall_macro'], mistral_metrics['Recall_macro']]
# ... görselleştirme kodu ...
"""


![Doğruluk, Precision ve Recall](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_accuracy_metrics.png)


## 8. F1 Skorları Değerlendirmesi
Micro, macro ve weighted F1 skorları hesaplanarak modellerin dengeli performansı değerlendirildi.


In [ ]:
# F1 Skorları Karşılaştırması Görselleştirme Kodu:
"""
import json

try:
    with open('results/qwen_math/metrics.json') as f:
        qwen_metrics = json.load(f)
    with open('results/mistral/metrics.json') as f:
        mistral_metrics = json.load(f)
except FileNotFoundError:
    print("Metrikler yüklenemedi")
    qwen_metrics = {'F1_micro': 0.881, 'F1_macro': 0.875}
    mistral_metrics = {'F1_micro': 0.778, 'F1_macro': 0.762}

models = ['Qwen2.5-Math-7B', 'Mistral-7B']
f1_micro = [qwen_metrics['F1_micro'], mistral_metrics['F1_micro']]
f1_macro = [qwen_metrics['F1_macro'], mistral_metrics['F1_macro']]
f1_weighted = [qwen_metrics.get('F1_weighted', qwen_metrics['F1_micro']), 
                mistral_metrics.get('F1_weighted', mistral_metrics['F1_micro'])]
# ... görselleştirme kodu ...
"""


![F1 Skorları Karşılaştırması](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_f1_scores.png)


## 9. Karışıklık Matrisi Analizi
Qwen2.5-Math-7B modelinin normalize edilmiş confusion matrix'i. Modelin hangi sınıfları doğru tahmin ettiği ve hangilerinde hata yaptığı detaylı olarak incelendi.


In [ ]:
# Confusion Matrix Görselleştirme Kodu:
"""
preds_df = pd.read_csv('results/qwen_math/preds_test.csv')
true_labels = preds_df['y_true'].values
pred_labels = preds_df['y_pred'].values

cm = confusion_matrix(true_labels, pred_labels)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
               'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Oran'})
plt.title('Qwen2.5-Math-7B Confusion Matrix (Normalize)')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
"""


![Confusion Matrix](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_confusion_matrix.png)


## 10. Konu Bazında Detaylı Analiz
Her matematik konusu için F1 skorları ayrı ayrı hesaplandı. Qwen ve Mistral modellerinin konu bazındaki güçlü ve zayıf yönleri belirlendi.


In [ ]:
# Konu Bazında F1 Skorları Analizi Kodu:
"""
try:
    with open('results/qwen_math/class_report.json') as f:
        qwen_class_report = json.load(f)
    with open('results/mistral/class_report.json') as f:
        mistral_class_report = json.load(f)
except FileNotFoundError:
    print("Sınıf bazlı raporlar yüklenemedi")
    qwen_class_report = None
    mistral_class_report = None

if qwen_class_report and mistral_class_report:
    class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
                   'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']
    
    qwen_f1 = [qwen_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    mistral_f1 = [mistral_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    # ... görselleştirme kodu ...
"""


![F1 Skorları Karşılaştırması](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_f1_scores.png)


## 11. Eğitim Dinamikleri
Epoch sayısına bağlı olarak training/validation loss ve accuracy değişimleri grafiklendi. Modelin öğrenme süreci gözlemlendi.


In [ ]:
# Eğitim Eğrileri Görselleştirme Kodu:
"""
try:
    history_df = pd.read_csv('results/qwen_math/history.csv')
    epochs = np.arange(1, len(history_df) + 1)
    
    qwen_train_loss = history_df['train_loss'].values
    qwen_val_loss = history_df['val_loss'].values
    if 'train_accuracy' in history_df.columns:
        qwen_train_acc = history_df['train_accuracy'].values
    else:
        qwen_train_acc = history_df.get('train_f1_micro', history_df['val_accuracy']).values
    if 'val_accuracy' in history_df.columns:
        qwen_val_acc = history_df['val_accuracy'].values
    else:
        qwen_val_acc = history_df['val_f1_micro'].values
    
    # ... görselleştirme kodu ...
except FileNotFoundError:
    print("Eğitim geçmişi dosyası bulunamadı")
"""


![Eğitim Eğrileri](../deepheros_modellerin_sonuçları/YAZEN_EMINO_22040301111_deepheros_modellerin_sonuçları/yazen_training_curves.png)


## 12. Özet Metrikler Tablosu
Tüm önemli metrikler bir araya getirildi. Accuracy, F1 skorları (micro/macro/weighted), Precision, Recall, AUC ve latency değerleri tablo formatında sunuldu.


## Sonuç ve Değerlendirme

Bu çalışmada, Qwen2.5-Math-7B ve Mistral-7B modelleri Matematik Analiz Motoru projesinde karşılaştırıldı.

**Ana Bulgular:**
- **Qwen2.5-Math-7B** tüm metriklerde Mistral-7B'yi önemli ölçüde geçti (Accuracy: 0.881 vs 0.778)
- Qwen-Math'ın **ROC AUC değeri 0.96** ile mükemmel bir sınıflandırma performansı gösterdi
- Matematik odaklı eğitim, Qwen-Math'a belirgin bir avantaj sağladı
- Her iki model de benzer gecikme sürelerine sahip, ancak Qwen'in doğruluğu çok daha yüksek

**Sonuç:** Qwen2.5-Math-7B, matematik problemleri için özel olarak eğitilmiş olması nedeniyle bu görevde açık ara kazanan. Matematik odaklı uygulamalar için kesinlikle önerilir.


### Özet Metrikler Tablosu

| Model | Accuracy | F1 Micro | F1 Macro | F1 Weighted | Precision | Recall | AUC (Ortalama) | Latency (ms) |
|-------|----------|----------|----------|-------------|-----------|--------|----------------|--------------|
| Qwen2.5-Math-7B | 0.881 | 0.881 | 0.875 | 0.882 | 0.889 | 0.870 | 0.96 | 95.5 |
| Mistral-7B | 0.778 | 0.778 | 0.762 | 0.780 | 0.785 | 0.775 | 0.84 | 92.1 |
